# Assignment week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Micha de Groot & Kaj Meijer

__Student id(s)__ : 10434410 & 10509534

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

[Selfie](http://imgur.com/RiMa4YY)

 
In this task we will try to expand the information that we find in the infoboxes using WordNet.

We follow the idea described in the YAGO paper: a string like `a beautiful actress from Denmark` is chopped up in a pre-modifier, a head and a post modifier. Here the head obviously is `actress`. 

## The tasks

 We perfom the task on a spreadsheet containing information about almost 1M persons found in the english Wikipedia.
In particular we are interested in the  `description` column.

1. (20%) For each string in the `description` column do the following:
    1. Determine pre-modifier,  head and  post modifier: store these in new columns
2.  (20%) For each head, determine the list of all ancestors of that head in WordNet up to "person": store this list in a new column.
3. (20%) Use Hearst patterns to extract something like the `description` from the `rdf-schema#comment`
    * Add your version as a new column
    * Design a comparison of your work with the description column which outputs something like accuracy/precision/recall. You are free, as long as you do it well.
4. (20%) Use Hearst patterns plus whatever else you like to extract the nationatility of the person described.
    * Include a brief evaluation.
2(actually 5). (20%) We are interested in gender aspects. Can you answer these questions:
    1. Can we determine the gender from the person from the description and its expansion? 
        * E.g., we could say it is a female if it was actress, but probably not say it is a male if it was actor
    2. There are two columns indicating the gender of the persons. One is made based on the first name, the other on the 3rd person pronouns in the longer description. Both are fairly accurate (more than .9 precision). Can you find a couple of remarkable differences between males and females based on your extracted wordnet concepts?

## Data
Available at <http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz>
    
## Hints
* Do this in a python notebook, using pandas, and possibly the pattern module for parsing the description.
* Make a 10K version of the file and experiment with that first.

Opgave 5.1

It is possible to detemine the gender of the person from the description if it contains gender specific terms. Most of the gender specific terms are female terms like actress, where actor could be male of female. 

Opgave 5.2

!!! We need data for this. !!!

In [ ]:
# http://www.clips.ua.ac.be/pages/pattern-en#parser
>>> from pattern.en import parse
>>> print parse('a beautiful actress from Denmark walks')

In [ ]:
from pattern.en import pprint
pprint(parse('a beautiful actress from Denmark walks', lemmata=True))

In [ ]:
pprint(parse('Ingeborg Moen Borgerud  is a Norwegian lawyer businessperson and former politician for the Labour Party.'))

In [8]:
# read 

import pandas as pd
Pdf = pd.read_csv('http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz',
                 compression='gzip')
Pdf = Pdf[Pdf.columns[2:]]
print Pdf.shape

Pdf.head()

(969996, 7)


,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128,19
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247,41
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242,41
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88,16
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301,52


In [9]:
# add some columns to Pdf 

# in this simple example we add the number of tokens in the description as a new column

def telwoordjes(string):
    return len(string.split())

Pdf['Description_length']= Pdf.description.astype(str).apply(telwoordjes)

Pdf.Description_length.head()

0    4
1    2
2    5
3    2
4    6
Name: Description_length, dtype: int64

In [14]:
# save, gzip, and load again

%time Pdf.to_pickle('PersonDF_naam_sex_description_snippetEXTRA.pkl')

%ls -lh 'PersonDF_naam_sex_description_snippetEXTRA.pkl'
%time !gzip PersonDF_naam_sex_description_snippetEXTRA.pkl
%ls -lh PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
%time !gunzip PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
# read 

%time Pdf = pd.read_pickle('PersonDF_naam_sex_description_snippetEXTRA.pkl')

!rm PersonDF_naam_sex_description_snippetEXTRA.pkl

CPU times: user 3.96 s, sys: 736 ms, total: 4.7 s
Wall time: 4.72 s
-rw-r--r--@ 1 admin  staff   351M Jul 28 16:24 PersonDF_naam_sex_description_snippetEXTRA.pkl
CPU times: user 91.9 ms, sys: 26.2 ms, total: 118 ms
Wall time: 22.8 s
-rw-r--r--@ 1 admin  staff   124M Jul 28 16:24 PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
CPU times: user 8.9 ms, sys: 14.2 ms, total: 23.1 ms
Wall time: 1.95 s
CPU times: user 3.25 s, sys: 574 ms, total: 3.82 s
Wall time: 3.85 s


In [11]:
# inspect the full descriptions

for x in Pdf['rdf-schema#comment'][:10]:
    print x

Ingeborg Moen Borgerud (born 23 September 1949) is a Norwegian lawyer businessperson and former politician for the Labour Party.
Ingeborg Nilsson (1924 – 1995) was a Norwegian figure skater. She competed at the 1952 Winter Olympics in Oslo.She was Norwegian individual champion in 1953 and 1954 and in pairs in 1956 1957 1958 and 1959 together with partner Reidar Børjeson.
Ingeborg Norell (born 1727) was the first Finnish female to have received an official decoration and to have been the subject of a commemorative plaque. She was commemorated in recognition for her act which saved many lives in Tenala in 1780.
Ingeborg Pehrson (16 December 1886 - 11 April 1950) was a Danish stage and film actress.
Ingeborg Ella Pfüller (born January 1 1932) was a female track and field athlete from Argentina competing in the discus throw. She represented her native country at the 1952 Summer Olympics and won the gold medal in the women's discus throw event at the 1955 Pan American Games in Mexico City M